In [1]:
from pydap.client import open_url
from pydap.cas.urs import setup_session
import datetime as dt  # Python standard library datetime  module
import numpy as np
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, addcyclic, shiftgrid
import pandas as pd
from datetime import datetime
from datetime import timedelta

# Global Vars
BASE_URL = 'E:/DataForDS/NCALDAS_NOAH0125_D.A'
END_URL = '.002.txt'

In [2]:
#Read Fire data
data = pd.read_csv("C:/Fires/NotFiresMillion.csv")
data

,Column1,DISCOVERY_DATE,X_tile,Y_tile
0,5477,1/1/1992,33,340
1,7369,1/1/1992,69,60
2,8336,1/1/1992,44,275
3,11560,1/1/1992,60,333
4,19031,1/1/1992,124,406
...,...,...,...,...
999995,978239,12/25/2015,84,348
999996,979529,12/25/2015,51,241
999997,983109,12/25/2015,88,47
999998,991149,12/25/2015,115,26


In [3]:
import random

myDateFormat = '%m/%d/%Y'
lats = np.genfromtxt('NoahLats.csv', delimiter=',')
lons = np.genfromtxt('NoahLons.csv', delimiter=',')

In [4]:
#Dis = map(lambda x: datetime.strptime(x,myDateFormat),data['DISCOVERY_DATE'])
import gc
import timeit
import random
import math


start = timeit.default_timer()

i=0
oldDate = None
dataset = None


Tair_f= []
Tair_f_min= []    
Tair_f_max= []
Wind_f= []
DOY = []
Humidity = []
LATITUDE = []
LONGITUDE = []

for indexes,fire in data.iterrows():
    date = datetime.strptime(str(fire['DISCOVERY_DATE']),myDateFormat)
    year = str(date.year)
    month = '{:02d}'.format(date.month)
    day = '{:02d}'.format(date.day)
    lat_idx = fire['X_tile']
    lon_idx = fire['Y_tile']
    
    try:
        if (date != oldDate):
            if (dataset != None):
                dataset.close()
            FullURL = BASE_URL+year+month+day+END_URL
            dataset = Dataset(FullURL, 'r')
    
        q = dataset.variables['Qair_f'][:][0,lat_idx,lon_idx].data.tolist()
        p = dataset.variables['Psurf_f'][:][0,lat_idx,lon_idx].data.tolist()
        T = dataset.variables['Tair_f'][:][0,lat_idx,lon_idx].data.tolist()
        lat_cor = 25 + ((lat_idx)*0.125) + round(random.uniform(0.0, 0.125),8)
        lon_cor = -125 + ((lon_idx)*0.125) + round(random.uniform(0.0, 0.125),8)
        LATITUDE.append(lat_cor)
        LONGITUDE.append(lon_cor)
        DOY.append(date.timetuple().tm_yday)
        Tair_f.append(T)
        Tair_f_min.append(dataset.variables['Tair_f_min'][:][0,lat_idx,lon_idx].data.tolist())   
        Tair_f_max.append(dataset.variables['Tair_f_max'][:][0,lat_idx,lon_idx].data.tolist())
        Wind_f.append(dataset.variables['Wind_f'][:][0,lat_idx,lon_idx].data.tolist())
        Humidity.append((0.263 * p * q )/(math.exp((17.67* (T - 273.16 ))/(T-29.65))))
        
        i=i+1
        if (i%10000 == 0):
            stop = timeit.default_timer()
            print('Time: ', stop - start , (i/len(data))*100,"% is Done")
            start = timeit.default_timer()
        oldDate = date
    except:
        LATITUDE.append(np.NaN)
        LONGITUDE.append(np.NaN)
        DOY.append(np.NaN)
        Tair_f.append(np.NaN)
        Tair_f_min.append(np.NaN)   
        Tair_f_max.append(np.NaN)
        Wind_f.append(np.NaN)
        Humidity.append(np.NaN) 

data['LATITUDE']=LATITUDE
data['LONGITUDE']=LONGITUDE
data['Tair_f']=Tair_f
data['Tair_f_min']=Tair_f_min
data['Tair_f_max']=Tair_f_max
data['Wind_f']=Wind_f
data['DOY']=DOY
data['Humidity']=Humidity

data.to_csv('C:/Fires/MillionNoFires.csv',mode ='a',index=False)
data

Time:  45.07948570000008 1.0 % is Done
Time:  40.60168540000086 2.0 % is Done
Time:  40.78686920000109 3.0 % is Done
Time:  40.945646600001055 4.0 % is Done
Time:  41.4314360999997 5.0 % is Done
Time:  40.455952199999956 6.0 % is Done
Time:  41.48836329999904 7.000000000000001 % is Done
Time:  40.839197699999204 8.0 % is Done
Time:  39.36739510000007 9.0 % is Done
Time:  38.78936599999906 10.0 % is Done
Time:  39.54635520000011 11.0 % is Done
Time:  39.22052939999958 12.0 % is Done
Time:  40.56732440000087 13.0 % is Done
Time:  53.12130090000028 14.000000000000002 % is Done
Time:  72.98516449999988 15.0 % is Done
Time:  66.63182790000064 16.0 % is Done
Time:  64.88445780000075 17.0 % is Done
Time:  47.19306789999973 18.0 % is Done
Time:  42.71631549999984 19.0 % is Done
Time:  42.308024199999636 20.0 % is Done
Time:  41.87500100000034 21.0 % is Done
Time:  41.18114710000009 22.0 % is Done
Time:  40.608722599999965 23.0 % is Done
Time:  39.694297399999414 24.0 % is Done
Time:  40.135609

,Column1,DISCOVERY_DATE,X_tile,Y_tile,LATITUDE,LONGITUDE,Tair_f,Tair_f_min,Tair_f_max,Wind_f,DOY,Humidity
0,5477,1/1/1992,33,340,29.169934,-82.456608,289.930969,286.980011,294.639954,5.831874,1,81.361927
1,7369,1/1/1992,69,60,33.725938,-117.381160,281.986023,276.519989,288.909943,2.710759,1,52.148556
2,8336,1/1/1992,44,275,30.614240,-90.579917,282.978241,277.659943,291.229919,2.602119,1,83.247789
3,11560,1/1/1992,60,333,32.524549,-83.304082,279.764496,277.690033,281.990021,5.988439,1,82.557932
4,19031,1/1/1992,124,406,40.599409,-74.148395,273.290802,270.259979,277.759979,2.456336,1,61.488584
...,...,...,...,...,...,...,...,...,...,...,...,...
999995,978239,12/25/2015,84,348,35.614332,-81.492754,291.419098,288.809875,294.370056,2.628271,359,92.213115
999996,979529,12/25/2015,51,241,31.439502,-94.766376,294.612366,290.670105,300.490051,1.763187,359,88.085401
999997,983109,12/25/2015,88,47,36.123315,-119.041792,280.513031,277.040009,284.169952,2.922746,359,77.502681
999998,991149,12/25/2015,115,26,39.455980,-121.732765,279.107971,276.769989,283.469910,1.545947,359,79.458690


In [6]:
data = data.drop(columns=['Column1','X_tile','Y_tile','DISCOVERY_DATE'])
data

,LATITUDE,LONGITUDE,Tair_f,Tair_f_min,Tair_f_max,Wind_f,DOY,Humidity
0,29.169934,-82.456608,289.930969,286.980011,294.639954,5.831874,1,81.361927
1,33.725938,-117.381160,281.986023,276.519989,288.909943,2.710759,1,52.148556
2,30.614240,-90.579917,282.978241,277.659943,291.229919,2.602119,1,83.247789
3,32.524549,-83.304082,279.764496,277.690033,281.990021,5.988439,1,82.557932
4,40.599409,-74.148395,273.290802,270.259979,277.759979,2.456336,1,61.488584
...,...,...,...,...,...,...,...,...
999995,35.614332,-81.492754,291.419098,288.809875,294.370056,2.628271,359,92.213115
999996,31.439502,-94.766376,294.612366,290.670105,300.490051,1.763187,359,88.085401
999997,36.123315,-119.041792,280.513031,277.040009,284.169952,2.922746,359,77.502681
999998,39.455980,-121.732765,279.107971,276.769989,283.469910,1.545947,359,79.458690


In [7]:
arr = np.full((1,1000000), 0)
arr
data["IS_FIRE"] = arr[0]
data

,LATITUDE,LONGITUDE,Tair_f,Tair_f_min,Tair_f_max,Wind_f,DOY,Humidity,IS_FIRE
0,29.169934,-82.456608,289.930969,286.980011,294.639954,5.831874,1,81.361927,0
1,33.725938,-117.381160,281.986023,276.519989,288.909943,2.710759,1,52.148556,0
2,30.614240,-90.579917,282.978241,277.659943,291.229919,2.602119,1,83.247789,0
3,32.524549,-83.304082,279.764496,277.690033,281.990021,5.988439,1,82.557932,0
4,40.599409,-74.148395,273.290802,270.259979,277.759979,2.456336,1,61.488584,0
...,...,...,...,...,...,...,...,...,...
999995,35.614332,-81.492754,291.419098,288.809875,294.370056,2.628271,359,92.213115,0
999996,31.439502,-94.766376,294.612366,290.670105,300.490051,1.763187,359,88.085401,0
999997,36.123315,-119.041792,280.513031,277.040009,284.169952,2.922746,359,77.502681,0
999998,39.455980,-121.732765,279.107971,276.769989,283.469910,1.545947,359,79.458690,0


In [16]:
data = data[data['Tair_f'] != 0]

In [17]:
data.to_csv('C:/Fires/MillionNoFires.csv',mode ='a',index=False)